In [84]:
import re

<link href="style.css" rel="stylesheet"></link>
<main>
<article class="day-desc"><h2>--- Day 19: Aplenty ---</h2><p>The Elves of Gear Island are thankful for your help and send you on your way. They even have a hang glider that someone <a href="9">stole</a> from Desert Island; since you're already going that direction, it would help them a lot if you would use it to get down there and return it to them.</p>
<p>As you reach the bottom of the <em>relentless avalanche of machine parts</em>, you discover that they're already forming a formidable heap. Don't worry, though - a group of Elves is already here organizing the parts, and they have a <span title="This part sparks joy. This part sparks joy. This part ALSO sparks joy... I think we need a different system."><em>system</em></span>.</p>
<p>To start, each part is rated in each of four categories:</p>
<ul>
<li><code>x</code>: E<em>x</em>tremely cool looking</li>
<li><code>m</code>: <em>M</em>usical (it makes a noise when you hit it)</li>
<li><code>a</code>: <em>A</em>erodynamic</li>
<li><code>s</code>: <em>S</em>hiny</li>
</ul>
<p>Then, each part is sent through a series of <em>workflows</em> that will ultimately <em>accept</em> or <em>reject</em> the part. Each workflow has a name and contains a list of <em>rules</em>; each rule specifies a condition and where to send the part if the condition is true. The first rule that matches the part being considered is applied immediately, and the part moves on to the destination described by the rule. (The last rule in each workflow has no condition and always applies if reached.)</p>
<p>Consider the workflow <code>ex{x&gt;10:one,m&lt;20:two,a&gt;30:R,A}</code>. This workflow is named <code>ex</code> and contains four rules. If workflow <code>ex</code> were considering a specific part, it would perform the following steps in order:</p>
<ul>
<li>Rule "<code>x&gt;10:one</code>": If the part's <code>x</code> is more than <code>10</code>, send the part to the workflow named <code>one</code>.</li>
<li>Rule "<code>m&lt;20:two</code>": Otherwise, if the part's <code>m</code> is less than <code>20</code>, send the part to the workflow named <code>two</code>.</li>
<li>Rule "<code>a&gt;30:R</code>": Otherwise, if the part's <code>a</code> is more than <code>30</code>, the part is immediately <em>rejected</em> (<code>R</code>).</li>
<li>Rule "<code>A</code>": Otherwise, because no other rules matched the part, the part is immediately <em>accepted</em> (<code>A</code>).</li>
</ul>
<p>If a part is sent to another workflow, it immediately switches to the start of that workflow instead and never returns. If a part is <em>accepted</em> (sent to <code>A</code>) or <em>rejected</em> (sent to <code>R</code>), the part immediately stops any further processing.</p>
<p>The system works, but it's not keeping up with the torrent of weird metal shapes. The Elves ask if you can help sort a few parts and give you the list of workflows and some part ratings (your puzzle input). For example:</p>
<pre><code>px{a&lt;2006:qkq,m&gt;2090:A,rfg}
pv{a&gt;1716:R,A}
lnx{m&gt;1548:A,A}
rfg{s&lt;537:gd,x&gt;2440:R,A}
qs{s&gt;3448:A,lnx}
qkq{x&lt;1416:A,crn}
crn{x&gt;2662:A,R}
in{s&lt;1351:px,qqz}
qqz{s&gt;2770:qs,m&lt;1801:hdj,R}
gd{a&gt;3333:R,R}
hdj{m&gt;838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
</code></pre>

<p>The workflows are listed first, followed by a blank line, then the ratings of the parts the Elves would like you to sort. All parts begin in the workflow named <code>in</code>. In this example, the five listed parts go through the following workflows:</p>
<ul>
<li><code>{x=787,m=2655,a=1222,s=2876}</code>: <code>in</code> -&gt; <code>qqz</code> -&gt; <code>qs</code> -&gt; <code>lnx</code> -&gt; <code><em>A</em></code></li>
<li><code>{x=1679,m=44,a=2067,s=496}</code>: <code>in</code> -&gt; <code>px</code> -&gt; <code>rfg</code> -&gt; <code>gd</code> -&gt; <code><em>R</em></code></li>
<li><code>{x=2036,m=264,a=79,s=2244}</code>: <code>in</code> -&gt; <code>qqz</code> -&gt; <code>hdj</code> -&gt; <code>pv</code> -&gt; <code><em>A</em></code></li>
<li><code>{x=2461,m=1339,a=466,s=291}</code>: <code>in</code> -&gt; <code>px</code> -&gt; <code>qkq</code> -&gt; <code>crn</code> -&gt; <code><em>R</em></code></li>
<li><code>{x=2127,m=1623,a=2188,s=1013}</code>: <code>in</code> -&gt; <code>px</code> -&gt; <code>rfg</code> -&gt; <code><em>A</em></code></li>
</ul>
<p>Ultimately, three parts are <em>accepted</em>. Adding up the <code>x</code>, <code>m</code>, <code>a</code>, and <code>s</code> rating for each of the accepted parts gives <code>7540</code> for the part with <code>x=787</code>, <code>4623</code> for the part with <code>x=2036</code>, and <code>6951</code> for the part with <code>x=2127</code>. Adding all of the ratings for <em>all</em> of the accepted parts gives the sum total of <code><em>19114</em></code>.</p>
<p>Sort through all of the parts you've been given; <em>what do you get if you add together all of the rating numbers for all of the parts that ultimately get accepted?</em></p>
</article>

</main>


In [85]:
from dataclasses import dataclass
from typing import Callable


inp = """
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
"""


@dataclass(frozen=True, order=True, slots=True)
class Part:
    x: int
    m: int
    a: int
    s: int

    def sum(self) -> int:
        return self.x + self.m + self.a + self.s


def create_rule(inp: str) -> tuple[Callable[[Part], bool], str]:
    rule = inp.split(":")
    if len(rule) == 1:
        return lambda p: True, inp

    rule, ret = rule
    attribute = rule[0]
    value = int(rule[2:])
    op = int.__lt__ if rule[1] == "<" else int.__gt__
    rule: Callable[[Part], bool] = lambda p: op(p.__getattribute__(attribute), value)

    return rule, ret


def parse(
    inp: str,
) -> tuple[dict[str, list[tuple[Callable[[Part], bool], str]]], list[Part]]:
    workflows_inp, parts_inp = re.split(r"(?:\r?\n){2,}", inp)

    workflows = {}
    for workflow in workflows_inp.strip().splitlines():
        name, rules = workflow.split("{")
        rules = [create_rule(r) for r in rules[:-1].split(",")]
        workflows[name] = rules

    parts = [
        Part(*[int(i) for i in re.findall(r"[0-9]+", part)])
        for part in parts_inp.strip().splitlines()
    ]
    return workflows, parts


def process_part(
    part: Part, workflows: dict[str, list[tuple[Callable[[Part], bool], str]]]
) -> bool:
    nxt = "in"
    while nxt != "A" and nxt != "R":
        for f, nxt1 in workflows[nxt]:
            if f(part):
                nxt = nxt1
                break

    return nxt == "A"


def process(inp: str) -> int:
    workflows, parts = parse(inp)
    return sum(part.sum() for part in parts if process_part(part, workflows))


assert process(inp) == 19114

In [86]:
with open("../input/day19.txt") as f:
    print(process(f.read()))

401674


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>401674</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Even with your help, the sorting process <em>still</em> isn't fast enough.</p>
<p>One of the Elves comes up with a new plan: rather than sort parts individually through all of these workflows, maybe you can figure out in advance which combinations of ratings will be accepted or rejected.</p>
<p>Each of the four ratings (<code>x</code>, <code>m</code>, <code>a</code>, <code>s</code>) can have an integer value ranging from a minimum of <code>1</code> to a maximum of <code>4000</code>. Of <em>all possible distinct combinations</em> of ratings, your job is to figure out which ones will be <em>accepted</em>.</p>
<p>In the above example, there are <code><em>167409079868000</em></code> distinct combinations of ratings that will be accepted.</p>
<p>Consider only your list of workflows; the list of part ratings that the Elves wanted you to sort is no longer relevant. <em>How many distinct combinations of ratings will be accepted by the Elves' workflows?</em></p>
</article>

</main>


In [87]:
from math import prod


def process_II(r: str):
    r = r.strip().split(":")
    if len(r) == 1:
        return tuple(r)
    return (r[0][0], r[0][1], int(r[0][2:]), r[1])


def parse_II(inp: str):
    workflows_inp, _ = re.split(r"(?:\r?\n){2,}", inp)

    workflows = {}
    for workflow in workflows_inp.strip().splitlines():
        name, rules = workflow.split("{")
        rules = [process_II(r) for r in rules[:-1].split(",")]
        workflows[name] = rules

    return workflows


def split_ranges(
    rule: str, ranges: dict[chr, tuple[int, int]]
) -> tuple[dict[chr, tuple[int, int]], dict[chr, tuple[int, int]]]:
    field, op, value, _ = rule
    succes, fail = dict(ranges), dict(ranges)
    l, h = ranges[field]
    if op == ">":
        succes[field] = max(l, value + 1), h
        fail[field] = l, min(value, h)
    else:
        succes[field] = l, min(h, value - 1)
        fail[field] = max(value, l), h
    return succes, fail


def possible_combinations(inp: str) -> int:
    def dfs(cur_wf: str, ranges: dict[chr, tuple[int, int]]) -> int:
        if cur_wf == "R":
            return 0
        if cur_wf == "A":
            return prod(h - l + 1 for l, h in ranges.values())

        combs = 0
        for rule in workflows[cur_wf]:
            next_wf = rule[-1]
            if len(rule) == 1:
                combs += dfs(next_wf, ranges)
            else:
                succes, fail = split_ranges(rule, ranges)
                combs += dfs(next_wf, succes)
                ranges = fail

        return combs

    workflows = parse_II(inp)
    return dfs("in", {c: (1, 4000) for c in "xmas"})


assert possible_combinations(inp) == 167409079868000

In [88]:
with open("../input/day19.txt") as f:
    print(possible_combinations(f.read()))

134906204068564


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>134906204068564</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2023">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="19/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Aplenty%22+%2D+Day+19+%2D+Advent+of+Code+2023&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2023%2Fday%2F19&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Aplenty%22+%2D+Day+19+%2D+Advent+of+Code+2023+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2023%2Fday%2F19';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
